# Google Summer of Code 2022: Train a DL model for synthetic data generation for model optimization

This notebook performs the [default quantization](https://docs.openvino.ai/latest/pot_default_quantization_usage.html#doxid-pot-default-quantization-usage) method of OpenVINO's Post-training Optimization Tool on a range of computer vision models as part of the GSoC22 project "Train a DL model for synthetic data generation for model optimization", which is implemented under the auspices of Intel's OpenVINO Toolkit organization. The performance of the models is then evaluated on a classification task on the CIFAR-10 test set. The selected CV models are pre-trained on CIFAR-10 and obtained from the [chenyaofo/pytorch-cifar-models](https://github.com/chenyaofo/pytorch-cifar-models) repository on GitHub.

## Imports

In [ ]:
import json
import sys
import time
from pathlib import Path

import ipywidgets as widgets
import numpy as np
import PIL
import torch
from IPython.display import Markdown, display
from matplotlib import pyplot as plt
from openvino.runtime import Core
from openvino.tools.pot import (IEEngine, compress_model_weights,
                                create_pipeline, load_model, save_model)
from openvino.tools.pot.api import DataLoader
from sklearn.metrics import accuracy_score
from torchvision import transforms
from torchvision.datasets import CIFAR10, ImageFolder


## Load the data
Load the four different datasets:
* Official CIFAR-10 training set
* FakeCIFAR generated by the StyleGAN2-ADA model
* FakeCIFAR generated by the DiStyleGAN model
* Fractal Images generated using the model from [Datumaro's repository](https://github.com/openvinotoolkit/datumaro) on GitHub

and create a DataLoader.

In [ ]:
datasets = {}

### DataLoader

In [ ]:
class CIFARLoader(DataLoader):
    """
    DataLoader for image data that is stored in a directory per category.
    """

    def __init__(self, data_source) -> None:
        """
        - data_source: dataset for which to create loader
        """
        self.dataset = data_source
        self.class_names = self.dataset.classes

    def __len__(self) -> int:
        """
        Returns the number of elements in the dataset
        """
        return len(self.dataset)

    def __getitem__(self, index: int) -> tuple:
        """
        Get item from self.dataset at the specified index.
        Returns (annotation, image), where annotation is a tuple (index, class_index)
        and image a preprocessed image in network shape
        """
        if index >= len(self):
            raise IndexError
        image, annotation = self.dataset[index]
        return (index, annotation), torch.unsqueeze(image, dim=0).numpy()

### Transform

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

### CIFAR10

In [ ]:
datasets['CIFAR10'] = CIFAR10(".",
    download=True, transform=transform)

cifar10_ds_test =  CIFAR10(".", train=False,
    download=True, transform=transform)

### FakeCIFAR StyleGAN2-ADA

In [ ]:
datasets['StyleGAN2-ADA'] = ImageFolder("datasets/StyleGAN2-ADA/", transform)

### FakeCIFAR DistyleGAN

In [ ]:
datasets['DiStyleGAN'] = ImageFolder("datasets/DiStyleGAN/", transform)

### Fractal Images

In [ ]:
datasets['Fractal'] = ImageFolder("datasets/Fractal/", transform)

## Load the PyTorch model
Select the PyTorch model to be 8-bit quantized.

In [ ]:
selection = widgets.Dropdown(
    options=['ResNet20', 'VGG16', 'MobileNetV2', 'ShuffleNetV2', 'RepVGG'],
    value='ResNet20',
    description='Model:',
    disabled=False,
)
display(selection)

Load the model from torch.hub

In [ ]:
model_names = {
    'ResNet20' : 'resnet20',
    'VGG16' : 'vgg16_bn',
    'MobileNetV2' : 'mobilenetv2_x1_4',
    'ShuffleNetV2' : 'shufflenetv2_x2_0',
    'RepVGG' : 'repvgg_a2'
} 

model_name = model_names[selection.value]
model = torch.hub.load("chenyaofo/pytorch-cifar-models", f"cifar10_{model_name}", pretrained=True)
model.eval()

## Evaluate the PyTorch model

In [ ]:
start = time.perf_counter()
predictions = []
true_labels = []
with torch.no_grad():
    for img, _class in cifar10_ds_test:
        img = img.unsqueeze(dim=0)
        res = model(img)
        predictions.append(int(np.argmax(res)))
        true_labels.append(_class)

end = time.perf_counter()
time_ir = end - start
print(
    f"Optimized model in Inference Engine/CPU: {time_ir/len(cifar10_ds_test):.3f} "
    f"seconds per image, FPS: {len(cifar10_ds_test)/time_ir:.2f}"
)

In [ ]:
accuracy_score(true_labels, predictions)

## Settings

In [ ]:
directory = Path("optimizations")
directory.mkdir(exist_ok=True)

save = directory / Path(f"{selection.value}")

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
onnx_path = save.with_suffix(".onnx")
ir_path = save.with_suffix(".xml")

## ONNX conversion

In [ ]:
if not onnx_path.exists():
    dummy_input = torch.randn(1, 3, 32, 32)
    
    torch.onnx.export(
        model,
        (dummy_input, ),
        onnx_path,
        opset_version=11
    )
    print(f"ONNX model exported to {onnx_path}.")
else:
    print(f"ONNX model {onnx_path} already exists.")

## Convert ONNX to OpenVINO IR format

In [ ]:
# Construct the command for Model Optimizer
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --output_dir "{save.parent}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

In [ ]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes.")
    mo_result = %sx $mo_command
    print("\n".join(mo_result))
else:
    print(f"IR model {ir_path} already exists.")

## Evaluate IR model

In [ ]:
# Load the optimized model and get the names of the input and output layer
ie = Core()
model_pot = ie.read_model(model=f"optimizations/{selection.value}.xml")
compiled_model_pot = ie.compile_model(model=model_pot, device_name="CPU")
input_layer = compiled_model_pot.input(0)
output_layer = compiled_model_pot.output(0)

In [ ]:
start = time.perf_counter()
predictions = []
true_labels = []
for img, _class in cifar10_ds_test:
    img = img.unsqueeze(dim=0)
    res = compiled_model_pot([img])[output_layer]
    predictions.append(int(np.argmax(res)))
    true_labels.append(_class)

end = time.perf_counter()
time_ir = end - start
print(
    f"Optimized model in Inference Engine/CPU: {time_ir/len(cifar10_ds_test):.3f} "
    f"seconds per image, FPS: {len(cifar10_ds_test)/time_ir:.2f}"
)

In [ ]:
accuracy_score(true_labels, predictions)

## Quantization

### Select calibration dataset

In [ ]:
calibration = widgets.Dropdown(
    options=['CIFAR10', 'StyleGAN2-ADA', 'DiStyleGAN', 'Fractal'],
    value='CIFAR10',
    description='Model:',
    disabled=False,
)
display(calibration)

### Optimize

In [ ]:
path_to_bin = ir_path.with_suffix('.bin')

# Model config specifies the model name and paths to model .xml and .bin file
model_config = {
    "model_name": "model",
    "model": ir_path,
    "weights": path_to_bin,
}

# Engine config
engine_config = {"device": "CPU"}

algorithms = [
    {
        "name": "DefaultQuantization",
        "params": {
            "target_device": "ANY"
        },
    }
]

# Step 1: Implement and create user's data loader
data_loader = CIFARLoader(datasets[calibration.value])

# Step 2: Load model
model = load_model(model_config=model_config)

# Step 3: Initialize the engine for metric calculation and statistics collection.
engine = IEEngine(config=engine_config, data_loader=data_loader)

# Step 4: Create a pipeline of compression algorithms and run it.
pipeline = create_pipeline(algorithms, engine)
compressed_model = pipeline.run(model=model)

# Step 5 (Optional): Compress model weights to quantized precision
#                     to reduce the size of the final .bin file.
compress_model_weights(compressed_model)

# Step 6: Save the compressed model to the desired path.
# Set save_path to the directory where the model should be saved
compressed_model_paths = save_model(
    model=compressed_model,
    save_path=save,
    model_name=f"optimized_model_{calibration.value}",
)

## Evaluate quantized model

In [ ]:
# Load the optimized model and get the names of the input and output layer
ie = Core()
model_pot = ie.read_model(model=f"optimizations/{selection.value}/optimized_model_{calibration.value}.xml")
compiled_model_pot = ie.compile_model(model=model_pot, device_name="CPU")
input_layer = compiled_model_pot.input(0)
output_layer = compiled_model_pot.output(0)

In [ ]:
start = time.perf_counter()
predictions = []
true_labels = []
for img, _class in cifar10_ds_test:
    img = img.unsqueeze(dim=0)
    res = compiled_model_pot([img])[output_layer]
    predictions.append(int(np.argmax(res)))
    true_labels.append(_class)

end = time.perf_counter()
time_ir = end - start
print(
    f"Optimized model in Inference Engine/CPU: {time_ir/len(cifar10_ds_test):.3f} "
    f"seconds per image, FPS: {len(cifar10_ds_test)/time_ir:.2f}"
)

In [ ]:
accuracy_score(true_labels, predictions)